In [1]:
from pymongo import MongoClient
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession \
    .builder \
    .appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/bigdata.raw") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/bigdata.raw") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1')\
    .getOrCreate()

In [3]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

Py4JJavaError: An error occurred while calling o50.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=127.0.0.1:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:179)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:136)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:94)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:249)
	at com.mongodb.client.internal.MongoClientDelegate$Delegate

In [ ]:
df = df.select("emoji", "sentence")
df.show()

In [ ]:
df.count()

## Preprocessing

In [ ]:
def divide_emoji(x):
    return x.split(" ")

def formed_(x):
    result = []
    for emoji in x[0]:
        result.append([emoji, x[1]])
    return result

In [ ]:
rdd = df.rdd.map(list)
rdd = rdd.map(lambda x: (divide_emoji(x[0]), x[1]))
rdd = rdd.flatMap(lambda x: formed_(x))
rdd.take(5)

In [ ]:
def separate_emoji(x):
    temp = x.split(',')
    return temp[0]

def separate_position(x):
    temp = x.split(',')
    return temp[1]

In [ ]:
rdd = rdd.map(lambda x: (separate_emoji(x[0]), int(separate_position(x[0])), x[1]))
rdd.take(5)

## For every emoji, find the top 10 words appears mostwhen using the emoji.

In [ ]:
def count_words(sentence):
    sentence = sentence.split(' ')
    words_frequency={}
    for word in sentence:
        word =''.join(ch for ch in word if ch.isalpha())
        word = word.lower()
        if word == '':
            continue
        if word in words_frequency:
            words_frequency[word] += 1
        else:
            words_frequency[word] = 1
    temp = list(words_frequency.items())
    temp.sort(key = lambda x: x[1], reverse=True)
    result = []
    num_top_word = 10
    if len(temp) < 10:
        num_top_word = len(temp)
    for i in range(num_top_word):
        result.append(temp[i][0])
    return result

In [ ]:
temp_rdd = rdd.map(lambda x: (x[0], x[2]))
group_by_emoji = temp_rdd.groupBy(lambda x: x[0])

In [ ]:
operate_rdd = group_by_emoji.map(lambda x: (x[0], list(x[1])))
# operate_rdd.take(1)

In [ ]:
def combine_string(x):
    all_sentence = ""
    for item in x:
        sentence = item[1].split(item[0])
        for word in sentence:
            if word != item[0]:
                all_sentence = all_sentence + word + " "
    return all_sentence

In [ ]:
combined = operate_rdd.map(lambda x: (x[0], combine_string(x[1])))
result = combined.map(lambda x: (x[0], count_words(x[1])))
result.take(1)

## Find which emoji used most with words begin with letter A, B, C...Z (alphabet order)

In [ ]:
def get_letter_stat(x):
    letters = {}
    words = x.split(' ')
    for word in words:
        word =''.join(ch for ch in word if ch.isalpha())
        if word == '':
            continue
        word = word.lower()
        if word[0] not in letters:
            letters[word[0]] = 1
        else:
            letters[word[0]] += 1
    temp = list(letters.items())
    temp.sort(key = lambda x: x[1], reverse=True)
    return temp[0]

In [ ]:
top_letter_for_emoji = combined.map(lambda x: (x[0], get_letter_stat(x[1])))

In [ ]:
def get_the_top(x):
    x.sort(key = lambda x: x[1], reverse=True)
    return x[0][0]

In [ ]:
key_is_letter = top_letter_for_emoji.map(lambda x: (x[1][0], (x[0], x[1][1])))
key_is_letter.take(3)

In [ ]:
group_by_letter = key_is_letter.groupBy(lambda x: x[0]).map(lambda x: (x[0], list(x[1])))
result_1 = group_by_letter.map(lambda x: (x[0], [row[1] for row in x[1]]))
result_1 = result_1.map(lambda x: (x[0], get_the_top(x[1])))
result_1.take(3)

## Find the most pair-wise frequency words and emojis

In [ ]:
def get_pair(x):
    pos = x[1]
    sentence = x[2]
    words = sentence.split(' ')
    word =''.join(ch for ch in words[pos-1] if ch.isalpha())
    word = word.lower()
    pair = (word, words[pos])
    return pair

def pair_frequency(x):
    pair_freq = {}
    emoji = x[0]
    pairs = x[1]
    for ele in pairs:
        if ele[1] != emoji or ele[0]=='':
            continue
        if ele in pair_freq:
            pair_freq[ele] += 1
        else:
            pair_freq[ele] = 1
    result = list(pair_freq.items())
    result.sort(key = lambda x: x[1], reverse=True)
    return result

In [ ]:
pairs = rdd.map(lambda x: (x[0], get_pair(x)))
group_pairs = pairs.groupBy(lambda x: x[0])
group_pairs = group_pairs.map(lambda x: (x[0], list(x[1])))
temp = group_pairs.map(lambda x: (x[0], [row[1] for row in x[1]]))
result_2 = temp.map(lambda x: (x[0], pair_frequency(x)))
result_2.take(1)